In [12]:
import random
import numpy as np

# Define the distance matrix for the cities
distance_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

# Define parameters for the ACO algorithm
num_ants = 5
num_iterations = 100
evaporation_rate = 0.5
alpha = 1.0
beta = 2.0
initial_pheromone_level = 1.0

# Initialize pheromone trails
pheromone_matrix = np.full((len(distance_matrix), len(distance_matrix)), initial_pheromone_level)

def get_path_distance(path):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distance_matrix[path[i]][path[i + 1]]
    total_distance += distance_matrix[path[-1]][path[0]]  # Return to the starting city
    return total_distance

def select_next_city(current_city, visited_cities):
    unvisited_cities = [city for city in range(len(distance_matrix)) if city not in visited_cities]
    probabilities = []
    total = 0
    for city in unvisited_cities:
        pheromone = pheromone_matrix[current_city][city]
        distance = distance_matrix[current_city][city]
        total += (pheromone ** alpha) * ((1.0 / distance) ** beta)
        probabilities.append(total)
    selected_city = None
    rand = random.uniform(0, total)
    for i, probability in enumerate(probabilities):
        if rand <= probability:
            selected_city = unvisited_cities[i]
            break
    return selected_city

def update_pheromones(pheromone_matrix, ant_paths, evaporation_rate):
    pheromone_matrix *= (1 - evaporation_rate)  # Evaporate pheromones
    for path_distance, path in ant_paths:
        for i in range(len(path) - 1):
            pheromone_matrix[path[i]][path[i + 1]] += 1.0 / path_distance
        pheromone_matrix[path[-1]][path[0]] += 1.0 / path_distance  # Return to the starting city

def ant_colony_optimization(num_ants, num_iterations):
    global pheromone_matrix
    best_path = None
    best_distance = float('inf')
    for _ in range(num_iterations):
        ant_paths = []
        for ant in range(num_ants):
            current_city = random.randint(0, len(distance_matrix) - 1)
            visited_cities = [current_city]
            while len(visited_cities) < len(distance_matrix):
                next_city = select_next_city(current_city, visited_cities)
                if next_city is None:
                    break
                visited_cities.append(next_city)
                current_city = next_city
            if len(visited_cities) == len(distance_matrix):
                path_distance = get_path_distance(visited_cities)
                ant_paths.append((path_distance, visited_cities))
                if path_distance < best_distance:
                    best_distance = path_distance
                    best_path = visited_cities
        update_pheromones(pheromone_matrix, ant_paths, evaporation_rate)
    return best_path, best_distance

best_path, best_distance = ant_colony_optimization(num_ants, num_iterations)
print("Best path:", best_path)
print("Best distance:", best_distance)

Best path: [1, 0, 2, 3]
Best distance: 80
